# Creating our own Auto Differentiation (AutoGrad) framework

In this practical exercise we will build our own, very simple, Auto-differentiation (or AutoGrad) framework.

## Coding the framework

### Step 1: Define a class for our variables

The key idea is that we will define our own class of `Variable` which is basically the same as a scalar (a number). So our class is created by passing it a `value`, and it stores this value internally.

But apart from being a placeholder for a number, we also want to keep track of the way every variable was created.

For example, if a variable $c$ is the result of the addition of two variables $a$ and $b$: $c = a + b$, then we would say that $a$ and $b$ are "parent" variables of $c$, and $c$ is their "child". The way $c$ was created was by adding these two parent variables together.

So apart from the value of the variable, we will also have to keep track of the parents, and on how each of them "contributes" to the creation of the variable - this is described by the local derivative associated with each of the parents, that tells us how a change in each of the parent variables translates into a change in the child variable.

This is important in order to implement our backwards pass. Each parent defines a "route" through which the gradients coming into this variable will have to flow through. So we will define a list of `gradRoutes` that will contain the list of parent variables and their corresponding local derivatives. A variable created directly (not resulting by any operation over existing variables) will have an empty `gradRoutes`.

Finally, we want each variable of ours to keep also the value of gradient of the quantity we are interested in with respect to that variable. We will create a placeholder for that as well, called `grad`. As seen before, this will accummulate the gradients that are backpropagated from the children of this variable when we implement the backpropagation algorithm. So we will initialise it to zero.

In [2]:
import numpy as np

In [3]:
class Variable: #Simple variable. These are the leafs of our tree, they can request to have a gradient calculated, or not
    def __init__(self, value):
        self.value = value
        self.gradRoutes = [] # A variable by default has no grad Routes (was not created by anything, just defined)
        self.grad = 0.0        
        
    def __str__(self):
        return 'Value: {self.value}'.format(self=self)        

Apart from the `__init__()` function which stores the value passed to our class and initialises the `gradRoutes` and `grad` member variables, we have also overloaded the function that python uses to convert a class into a string representation: `__str__()`. This will allow us to print our class.

We cannot do much yet with this class, apart from storing values into our variables and printing them out. Let's try this out. 

In [4]:
a = Variable(4.3)
b = Variable(5.2)
print(a)
print(b)

Value: 4.3
Value: 5.2


### Step 2: Define operations over our variables

The next step would be to implement operations on our variables. Let's first define the operations for addition and multiplication.

These will be functions that take two variables as input and produce a new (child) variable with a value equal to the sum or the product of the two inputs. Apart from the forward pass though, we should keep track of how this new variable was created: the two parent variables, and their corresponding local derivatives.

In [5]:
def vAdd(A: Variable, B: Variable): # Addition
    result = Variable(A.value + B.value) # Create a new Variable to store the result, and pass it the value = a + b
          
    #keep track of the parent variables, and of the local derivative associated with each one
    result.gradRoutes.append((A, 1)) # dresult / dA = 1
    result.gradRoutes.append((B, 1)) # dresult / dB = 1
    
    return result
    
def vMul(A: Variable, B: Variable): # Addition
    result = Variable(A.value * B.value) # Create a new Variable to store the result, and pass it the value = a * b
          
    #keep track of the parent variables, and of the local derivative associated with each one
    result.gradRoutes.append((A, B.value)) # dresult / dA = B
    result.gradRoutes.append((B, A.value)) # dresult / dB = A
    
    return result

So for example to calculate $d = (a + b) * c$ we first need to calculate $(a + b)$ and then mutiply the result with $c$

In [6]:
a = Variable(2) # a = 2
b = Variable(3) # b = 3
c = Variable(4) # c = 4

#d = (a + b) * c = 20
d = vMul(vAdd(a, b), c)

print(d)

Value: 20


### Step 3: Implement the backpropagation function

The final step is to implement the backpropagation function. This starts with a child variable, and backpropagates gradients through the routes defined recursively. It uses the two rules that we saw before:

- Accumulate the incoming gradients from the different grad routes that lead to a node (a variable). Each of the incoming gradients describe a different way in which the node affects the quantity of interest, so this sum will be the final gradient for the node
- Multiply every incoming gradient with each of the local derivatives corresponding to parent variables (this would be the application of the chain rule), and continue the backpropagation through the corresponding route (for each of the parent variables)

We update the `Variable` class accordingly. We also update the `__str___()` function to include also gradient information.

In [7]:
class Variable: #Simple variable. These are the leafs of our tree, they can request to have a gradient calculated, or not
    def __init__(self, value):
        self.value = value
        self.gradRoutes = [] # A variable by default has no grad Routes (was not created by anything, just defined)
        self.grad = 0.0
    
    def backProp(self, route_val = 1.0):
        # Add together the incoming gradients from the different routes that lead to a node - this will be the final gradient for the node
        self.grad += route_val
                
        # For every parent variable and corresponding local derivative value that we have in the gradRoutes, continue with the gradient calculation
        for variable, local_derivative_value in self.gradRoutes:
            # Multiply the incoming gradient with the local derivatives corresponding the parent variable, and continue the backpropagation
            variable.backProp(local_derivative_value * route_val)
    
    def __str__(self):
        return 'Value: {self.value}, Gradient: {self.grad}'.format(self=self)

<font color=blue>**Question:** Why did we set the default value of route_val equal to 1.0?</font>

---

Because that is the original node from which we are starting to calculate. The derivative of that node by itself is one.

---


This should be all. If we want to calculate the derivative of the result with respect to any of the variables that participated in the calculation, we just need to call backprop on the result, and then read the derivatives out.

In [8]:
a = Variable(2)           # a = 2
b = Variable(3)           # b = 3
c = Variable(4)           # c = 4
res = vMul(vAdd(a, b), c) # res = (a + b) * c = 20

print("Result =", res.value)

# Call backprop on the result
res.backProp()

# Now all variables should contain in their "grad" the derivative d(res) / d(variable)
print("The derivative of the result with respect to a is:", a.grad)
print("The derivative of the result with respect to b is:", b.grad)
print("The derivative of the result with respect to c is:", c.grad)

Result = 20
The derivative of the result with respect to a is: 4.0
The derivative of the result with respect to b is: 4.0
The derivative of the result with respect to c is: 5.0


In the following example, variable $a$ affects the result through two different routes

In [9]:
a = Variable(4)  # a = 4
b = Variable(3)  # b = 3
c = vAdd(a, b)   # c = 4 + 3
res = vMul(a, c) # res = a * c = 28

print("Result =", res.value)

# Call backprop on the result
res.backProp()

# Now all variables should contain in their "grad" the derivative d(res) / d(variable)
print("The derivative of the result with respect to a is:", a.grad)
print("The derivative of the result with respect to b is:", b.grad)
# Also for intermediate results
print("The derivative of the result with respect to c is:", c.grad)

Result = 28
The derivative of the result with respect to a is: 11.0
The derivative of the result with respect to b is: 4.0
The derivative of the result with respect to c is: 4.0


<font color=blue>**Question:** Can you now use this setup to calculate the derivative of $c$ with respect to $a$ and $b$?</font>

In [10]:
#first reset the gradients
a.grad = 0
b.grad = 0
c.grad = 0
c.backProp()
print("The derivative of c with respect to a is:", a.grad)
print("The derivative of c with respect to b is:", b.grad)

The derivative of c with respect to a is: 1.0
The derivative of c with respect to b is: 1.0


## Final touches

If you understood how this works up to here, then you should be already good to go. But since we want to use our auto grad to do some practical work, we will continue working on it a bit, to make it a bit more usable and complete it with more operations. Many of the subsequent steps are quite "engineering" in nature.

### Improving usability: overloading operators

Of course this is still highly incomplete, very inefficient and not very usable. Lets first improve a usability issue. Instead of having to call different functions for the operations like `res = vMul(a, c)`, we would like to be able to directly write them down like `res = a * b`. To achieve this, we should overload [Python's special functions for operator overloading](https://docs.python.org/3/reference/datamodel.html#emulating-numeric-types).

Here's how to do this for the addition and multiplication.

In [11]:
class Variable: #Simple variable. These are the leafs of our tree, they can request to have a gradient calculated, or not
    def __init__(self, value):
        self.value = value
        self.gradRoutes = [] # A variable by default has no grad Routes (was not created by anything, just defined)
        self.grad = 0.0
    
    def backProp(self, route_val = 1.0):
        # Add together the incoming gradients from the different routes that lead to a node - this will be the final gradient for the node
        self.grad += route_val
                
        # For every parent variable and corresponding local derivative value that we have in the gradRoutes, continue with the gradient calculation
        for variable, local_derivative_value in self.gradRoutes:
            # Multiply the incoming gradient with the local derivatives corresponding the parent variable, and continue the backpropagation
            variable.backProp(local_derivative_value * route_val)
            
    def __add__(self, b):
        return vAdd(self, b)
        
    def __mul__(self, b):
        return vMul(self, b)            
    
    def __str__(self):
        return 'Value: {self.value}, Gradient: {self.grad}'.format(self=self)

In [12]:
a = Variable(4)  # a = 4
b = Variable(3)  # b = 3
c = a + b        # c = 4 + 3
res = a * c      # res = a * c = 28

print("Result =", res.value)

# Call backprop on the result
res.backProp()

# Now all variables should contain in their "grad" the derivative d(res) / d(variable)
print("The derivative of the result with respect to a is:", a.grad)
print("The derivative of the result with respect to b is:", b.grad)
# Also for intermediate results
print("The derivative of the result with respect to c is:", b.grad)

Result = 28
The derivative of the result with respect to a is: 11.0
The derivative of the result with respect to b is: 4.0
The derivative of the result with respect to c is: 4.0


### Zeroing gradients

A last thing to note is that once we call `backProp`, our gradients are calculated and our variables are now "dirty" in the sense that if we call backprop again, the new result will be added to the previous one:

In [13]:
a = Variable(4)   # a = 4
b = Variable(3)   # b = 3
res = (a + b) * a # res = a * c = 28

# Call backprop on the result
res.backProp()
print("The derivative of the result with respect to a is:", a.grad)
print("The derivative of the result with respect to b is:", b.grad)

# Call backprop on the result once more
print("Second time")
res.backProp()
print("The derivative of the result with respect to a is:", a.grad)
print("The derivative of the result with respect to b is:", b.grad)

The derivative of the result with respect to a is: 11.0
The derivative of the result with respect to b is: 4.0
Second time
The derivative of the result with respect to a is: 22.0
The derivative of the result with respect to b is: 8.0


This will actually turn out to be quite useful, e.g. when we want to accumulate weight gradients over different samples in our learning loops (see next week's notebook), but we need a way to control it.

To avoid this, we should reset the gradients to zero before we call `backProp` again. We can do it one by one for every variable, but we will also implement a function that does this recursively from the child node we backProped, all the way to the parents.

In [14]:
class Variable: #Simple variable. These are the leafs of our tree, they can request to have a gradient calculated, or not
    def __init__(self, value):
        self.value = value
        self.gradRoutes = [] # A variable by default has no grad Routes (was not created by anything, just defined)
        self.grad = 0.0
    
    def backProp(self, route_val = 1.0):
        # Add together the incoming gradients from the different routes that lead to a node - this will be the final gradient for the node
        self.grad += route_val
                
        # For every parent variable and corresponding local derivative value that we have in the gradRoutes, continue with the gradient calculation
        for variable, local_derivative_value in self.gradRoutes:
            # Multiply the incoming gradient with the local derivatives corresponding the parent variable, and continue the backpropagation
            variable.backProp(local_derivative_value * route_val)

    def zeroGrad(self):
        self.grad = 0.0
        
    def zeroGradsRecursively(self):
        self.zeroGrad()
        for variable, _ in self.gradRoutes:
            variable.zeroGradsRecursively()
            
    def __add__(self, b):
        return vAdd(self, b)
        
    def __mul__(self, b):
        return vMul(self, b)           
    
    def __str__(self):
        return 'Value: {self.value}, Gradient: {self.grad}'.format(self=self)

In [15]:
a = Variable(4)   # a = 4
b = Variable(3)   # b = 3
res = (a + b) * a # res = a * c = 28

# Call backprop on the result
res.backProp()
print("The derivative of the result with respect to a is:", a.grad)
print("The derivative of the result with respect to b is:", b.grad)

# Zero gradients
res.zeroGradsRecursively()

# Call backprop on the result once more
print("Second time")
res.backProp()
print("The derivative of the result with respect to a is:", a.grad)
print("The derivative of the result with respect to b is:", b.grad)

The derivative of the result with respect to a is: 11.0
The derivative of the result with respect to b is: 4.0
Second time
The derivative of the result with respect to a is: 11.0
The derivative of the result with respect to b is: 4.0


## More Improvements - Homework

There are number of ways we can improve our simple network. The most important is probably being able to work with vectors and matrices - we will not implement this ourselves though, next week we will see a framework that does this. For the time being, we will focus on other, simpler improvements.

---
### <font color=blue>Excercise 1 (Easy):</font>
    
<font color=blue>We usually do not require gradients for all our variables. If we could indicate which variables require gradients, then we could keep track of the routes that lead to these variables only and drop all the rest. This would be a huge improvement in resources and speed (number of calculations). Can you add this functionality?</font>

We keep track of each variables and the generated by the operators by having a depth level.

We'll also have a function that prints them all out

In [16]:
class Variable: #Simple variable. These are the leafs of our tree, they can request to have a gradient calculated, or not
    def __init__(self, value,level=0):
        self.value = value
        self.gradRoutes = [] # A variable by default has no grad Routes (was not created by anything, just defined)
        self.grad = 0.0
        self.lev = level
    
    def backProp(self, route_val = 1.0):
        # Add together the incoming gradients from the different routes that lead to a node - this will be the final gradient for the node
        self.grad += route_val
                
        # For every parent variable and corresponding local derivative value that we have in the gradRoutes, continue with the gradient calculation
        for variable, local_derivative_value in self.gradRoutes:
            # Multiply the incoming gradient with the local derivatives corresponding the parent variable, and continue the backpropagation
            variable.backProp(local_derivative_value * route_val)

    def print_nodes(self):
        print(self)
        for var, _ in self.gradRoutes:
            var.print_nodes()
        
    def target_backprop(self,vars:list[Variable],route_val = 1.0)->float:
        # Add together the incoming gradients from the different routes that lead to a node - this will be the final gradient for the node
        self.grad += route_val
        if self in vars:
            vars.pop(self)
        if(len(vars) == 0):
            return None
        # For every parent variable and corresponding local derivative value that we have in the gradRoutes, continue with the gradient calculation
        for variable, local_derivative_value in self.gradRoutes:
            # Multiply the incoming gradient with the local derivatives corresponding the parent variable, and continue the backpropagation
            variable.backProp(local_derivative_value * route_val)
            
    def zeroGrad(self):
        self.grad = 0.0
        
    def zeroGradsRecursively(self):
        self.zeroGrad()
        for variable, _ in self.gradRoutes:
            variable.zeroGradsRecursively()
            
    def __add__(self, b):
        return vAdd(self, b)
        
    def __mul__(self, b):
        return vMul(self, b)           
    
    def __str__(self):
        return 'level:{self.lev}, value: {self.value}, Gradient: {self.grad}'.format(self=self)

In [17]:
a = Variable(4)   # a = 4
b = Variable(3)   # b = 3
res = (a + b) * a # res = a * c = 28
res.print_nodes()

level:0, value: 28, Gradient: 0.0
level:0, value: 7, Gradient: 0.0
level:0, value: 4, Gradient: 0.0
level:0, value: 3, Gradient: 0.0
level:0, value: 4, Gradient: 0.0


To have the levels and index properly assigned we have to modify the operator functions:

In [18]:
def vAdd(A: Variable, B: Variable): # Addition
    result = Variable(A.value + B.value, level=A.lev+1) # Create a new Variable to store the result, and pass it the value = a + b
          
    #keep track of the parent variables, and of the local derivative associated with each one
    result.gradRoutes.append((A, 1)) # dresult / dA = 1
    result.gradRoutes.append((B, 1)) # dresult / dB = 1
    
    return result
    
def vMul(A: Variable, B: Variable): # Addition
    result = Variable(A.value * B.value, level=A.lev+1) # Create a new Variable to store the result, and pass it the value = a * b
          
    #keep track of the parent variables, and of the local derivative associated with each one
    result.gradRoutes.append((A, B.value)) # dresult / dA = B
    result.gradRoutes.append((B, A.value)) # dresult / dB = A
    
    return result

In [19]:
a = Variable(4)   # a = 4
b = Variable(3)   # b = 3
res = (a + b) * a # res = a * c = 28
res.backProp()
res.print_nodes()

level:2, value: 28, Gradient: 1.0
level:1, value: 7, Gradient: 4.0
level:0, value: 4, Gradient: 11.0
level:0, value: 3, Gradient: 4.0
level:0, value: 4, Gradient: 11.0


From this we can see that we can search for 

In [20]:
print(res.gradRoutes[1][0])

level:0, value: 4, Gradient: 11.0


In [21]:
def recursive_print_grads(Variable):
    print(Variable)
    for var, _ in Variable.gradRoutes:
        recursive_print_grads(var)

In [22]:
recursive_print_grads(res)

level:2, value: 28, Gradient: 1.0
level:1, value: 7, Gradient: 4.0
level:0, value: 4, Gradient: 11.0
level:0, value: 3, Gradient: 4.0
level:0, value: 4, Gradient: 11.0


In contrast with:

In [23]:
a = Variable(4)   # a = 4
b = Variable(3)   # b = 3
res = (a + b) * a # res = a * c = 28
res.backProp()
recursive_print_grads(res)

level:2, value: 28, Gradient: 1.0
level:1, value: 7, Gradient: 4.0
level:0, value: 4, Gradient: 11.0
level:0, value: 3, Gradient: 4.0
level:0, value: 4, Gradient: 11.0


---
### <font color=blue>Excercise 2 (Normal):</font>
    
<font color=blue>We obviously need to implement more functions - implement the following functions:
- Subtraction
- Raising to a power
- Division
- Unary negation
- The (natural) exponential function exp(x)
- ... any other function you might want</font>

In [24]:
import math

In [25]:
def vAdd(A: Variable, B: Variable): # Addition
    result = Variable(A.value + B.value) # Create a new Variable to store the result, and pass it the value = a + b
          
    #keep track of the parent variables, and of the local derivative associated with each one
    result.gradRoutes.append((A, 1)) # dresult / dA = 1
    result.gradRoutes.append((B, 1)) # dresult / dB = 1
    
    return result
    
def vMul(A: Variable, B: Variable): # Addition
    result = Variable(A.value * B.value) # Create a new Variable to store the result, and pass it the value = a * b
          
    #keep track of the parent variables, and of the local derivative associated with each one
    result.gradRoutes.append((A, B.value)) # dresult / dA = B
    result.gradRoutes.append((B, A.value)) # dresult / dB = A
    
    return result

def vSub(A: Variable, B: Variable): # Addition
    result = Variable(A.value - B.value) # Create a new Variable to store the result, and pass it the value = a + b
          
    #keep track of the parent variables, and of the local derivative associated with each one
    result.gradRoutes.append((A, 1)) # dresult / dA = 1
    result.gradRoutes.append((B, -1)) # dresult / dB = 1
    
    return result
    
def vPow(A: Variable, B: Variable): # Addition
    result = Variable(A.value**B.value) # Create a new Variable to store the result, and pass it the value = a * b
          
    #keep track of the parent variables, and of the local derivative associated with each one
    result.gradRoutes.append((A, B.value*A.value**(B.value-1))) # dresult / dA = B
    result.gradRoutes.append((B, (A.value**B.value)*math.log(B.value))) # dresult / dB = A
    
    return result

def vDiv(A: Variable, B: Variable): # Addition
    result = Variable(A.value / B.value) # Create a new Variable to store the result, and pass it the value = a * b
          
    #keep track of the parent variables, and of the local derivative associated with each one
    result.gradRoutes.append((A, 1/B.value)) # dresult / dA = B
    result.gradRoutes.append((B, (-1)*A.value/(B.value**2))) # dresult / dB = A
    
    return result

def vExp(A: Variable): # Addition
    result = Variable(math.exp(A.value)) # Create a new Variable to store the result, and pass it the value = a * b
          
    #keep track of the parent variables, and of the local derivative associated with each one
    result.gradRoutes.append((A,A.value*math.exp(A.value-1))) # dresult / dA = B
    
    return result 

def vNeg(A: Variable): # Addition
    result = Variable(A.value * -1) # Create a new Variable to store the result, and pass it the value = a * b
          
    #keep track of the parent variables, and of the local derivative associated with each one
    result.gradRoutes.append((A,-1)) # dresult / dA = B
    
    return result

In [26]:
class Variable: #Simple variable. These are the leafs of our tree, they can request to have a gradient calculated, or not
    def __init__(self, value):
        self.value = value
        self.gradRoutes = [] # A variable by default has no grad Routes (was not created by anything, just defined)
        self.grad = 0.0
    
    def backProp(self, route_val = 1.0):
        # Add together the incoming gradients from the different routes that lead to a node - this will be the final gradient for the node
        self.grad += route_val
                
        # For every parent variable and corresponding local derivative value that we have in the gradRoutes, continue with the gradient calculation
        for variable, local_derivative_value in self.gradRoutes:
            # Multiply the incoming gradient with the local derivatives corresponding the parent variable, and continue the backpropagation
            variable.backProp(local_derivative_value * route_val)

    def target_backprop(self,vars:list[Variable],route_val = 1.0)->float:
        # Add together the incoming gradients from the different routes that lead to a node - this will be the final gradient for the node
        self.grad += route_val
        if self in vars:
            vars.pop(self)
        if(len(vars) == 0):
            return None
        # For every parent variable and corresponding local derivative value that we have in the gradRoutes, continue with the gradient calculation
        for variable, local_derivative_value in self.gradRoutes:
            # Multiply the incoming gradient with the local derivatives corresponding the parent variable, and continue the backpropagation
            variable.backProp(local_derivative_value * route_val)
            

    def zeroGrad(self):
        self.grad = 0.0
        
    def zeroGradsRecursively(self):
        self.zeroGrad()
        for variable, _ in self.gradRoutes:
            variable.zeroGradsRecursively()
            
    def __add__(self, b):
        return vAdd(self, b)

    def __sub__(self, b):
        return vSub(self, b)
   
    def __mul__(self, b):
        return vMul(self, b) 
    
    def __truediv__(self, b):
        return vDiv(self, b)

    def __pow__(self, b):
        return vPow(self,b)         
    
    def __neg__(self):
        return vNeg(self)
        
    def __str__(self):
        return 'Value: {self.value}, Gradient: {self.grad}'.format(self=self)

Test it:

In [27]:
a = Variable(4)   # a = 4
b = Variable(3)   # b = 3
c = Variable(8)
e = Variable(2)
#res = a / b
res = vExp(((a - b + c) * a ) / (c**(e)))# res = a * c = 28
res.backProp()
recursive_print_grads(res)

Value: 1.7550546569602985, Gradient: 1.0
Value: 0.5625, Gradient: 0.3631772961156893
Value: 36, Gradient: 0.005674645251807645
Value: 9, Gradient: 0.02269858100723058
Value: 1, Gradient: 0.02269858100723058
Value: 4, Gradient: 0.07377038827349938
Value: 3, Gradient: -0.02269858100723058
Value: 8, Gradient: -0.028373226259038223
Value: 4, Gradient: 0.07377038827349938
Value: 64, Gradient: -0.0031919879541418
Value: 8, Gradient: -0.028373226259038223
Value: 2, Gradient: -0.14160111685086058


---
### <font color=blue>Excercise 3 (Difficult):</font>
    
<font color=blue>Our operations currently accept only instances of our Variable class as inputs. So, if you wanted to calculate `a = b * 2` where `b` is an instance of our variable class and `2` is just a numerical constant you would get an error as our framework does not know how to multiply a `Variable` with a number. You should instead write `a = b * Variable(2)` to achieve this.</font>

<font color=blue>Improve further the usability of our framework by allowing our functions to mix numbers and Variables in the same operation (look also into the overloads of the [reflected operands in python](https://docs.python.org/3/reference/datamodel.html#object.__radd__))</font>

*Hint: look at the `isinstance()` function*

*Hint: You might also want to look for reflected operators...*

In [35]:
class Variable: #Simple variable. These are the leafs of our tree, they can request to have a gradient calculated, or not
    def __init__(self, value):
        self.value = value
        self.gradRoutes = [] # A variable by default has no grad Routes (was not created by anything, just defined)
        self.grad = 0.0
    
    def backProp(self, route_val = 1.0):
        # Add together the incoming gradients from the different routes that lead to a node - this will be the final gradient for the node
        self.grad += route_val
                
        # For every parent variable and corresponding local derivative value that we have in the gradRoutes, continue with the gradient calculation
        for variable, local_derivative_value in self.gradRoutes:
            # Multiply the incoming gradient with the local derivatives corresponding the parent variable, and continue the backpropagation
            variable.backProp(local_derivative_value * route_val)

    def target_backprop(self,vars:list[Variable],route_val = 1.0)->float:
        # Add together the incoming gradients from the different routes that lead to a node - this will be the final gradient for the node
        self.grad += route_val
        if self in vars:
            vars.pop(self)
        if(len(vars) == 0):
            return None
        # For every parent variable and corresponding local derivative value that we have in the gradRoutes, continue with the gradient calculation
        for variable, local_derivative_value in self.gradRoutes:
            # Multiply the incoming gradient with the local derivatives corresponding the parent variable, and continue the backpropagation
            variable.backProp(local_derivative_value * route_val)
            

    def zeroGrad(self):
        self.grad = 0.0
        
    def zeroGradsRecursively(self):
        self.zeroGrad()
        for variable, _ in self.gradRoutes:
            variable.zeroGradsRecursively()
            
    def __add__(self, b): #called if left operand is of this class
        if isinstance(b,Variable):
            return vAdd(self, b)
        else:
            return vAdd(self, Variable(b))
    
    def __radd__(self, b): #called if right operand is of this class
        return vAdd(self, Variable(b))

    def __sub__(self, b):  #called if left operand is of this class
        if isinstance(b,Variable):
            return vSub(self, b)
        else:
            return vSub(self, Variable(b))
    
    def __rsub__(self, b): #called if right operand is of this class
        return vSub(self, Variable(b))

    def __mul__(self, b): 
        if isinstance(b,Variable):
            return vMul(self, b) 
        else:
            return vMul(self, Variable(b))

    def __rmul__(self, b): #called if right operand is of this class
        return vMul(self, Variable(b))

    def __truediv__(self, b):
        if isinstance(b,Variable):
            return vDiv(self, b)
        else:
            return vDiv(self, Variable(b))
    
    def __rtruediv__(self, b): #called if right operand is of this class
        return vDiv(self, Variable(b))

    def __pow__(self, b):
        if isinstance(b,Variable):
            return vPow(self,b)         
        else:
            return vPow(self, Variable(b))
    
    def __rpow__(self, b): #called if right operand is of this class
        return vPow(self, Variable(b))
    
    def __neg__(self):
        return vNeg(self)
        
    def __str__(self):
        return 'Value: {self.value}, Gradient: {self.grad}'.format(self=self)

Test:

In [29]:
a = Variable(4)   # a = 4
b = Variable(3)   # b = 3
c = Variable(8)
e = Variable(2)
res = 1 + a - b - 2
res.backProp()
recursive_print_grads(res)

Value: 0, Gradient: 1.0
Value: 2, Gradient: 1.0
Value: 5, Gradient: 1.0
Value: 4, Gradient: 1.0
Value: 1, Gradient: 1.0
Value: 3, Gradient: -1.0
Value: 2, Gradient: -1.0


In [30]:
res = vExp(((a - 4 + (6**c)) * -0.5 ) / (c**(3)))# res = a * c = 28
res.backProp()
recursive_print_grads(res)

Value: 6.616261056709485e-112, Gradient: 1.0
Value: -256.0, Gradient: -6.231005235677854e-110
Value: -131072.0, Gradient: -1.216993210093331e-112
Value: 262144, Gradient: 6.084966050466654e-113
Value: 0, Gradient: 6.084966050466654e-113
Value: 4, Gradient: 1.0
Value: 4, Gradient: -6.084966050466654e-113
Value: 262144, Gradient: 6.084966050466654e-113
Value: 8, Gradient: 5.98176502625074e-108
Value: 6, Gradient: 2.858102434261858e-107
Value: -0.5, Gradient: -3.1902746806670613e-107
Value: 512, Gradient: -3.115502617838927e-110
Value: 8, Gradient: 5.98176502625074e-108
Value: 3, Gradient: -1.7524374842037816e-107


---
### <font color=blue>Excercise 4 (Very Difficult):</font>
    
<font color=blue>Instead of keeping track of the local derivative value and doing the multiplication between the local derivative value and the incoming gradient explicitly, it is better to keep track of the actual function used to calculate the local derivative and combine it with the incoming one (the `route_val`). The idea is to keep a note of the recipe for calculating the local derivative value instead of the value itself.</font>

<font color=blue>In our case, since we deal with scalars, this function would basically boil down to doing this multiplication with the `route_val` (applying the chain rule), but when we vectorise our inputs and start dealing with tensors instead of scalars (and `route_val` becomes a matrix), this will become a bit more complicated (a dot product). So, keeping track of the function instead of the local derivative value will allow us to easily extend this framework to this scenario.</font>

<font color=blue>In addition, keeping a note of the function instead of the value, allows us to abstract away stuff. This basically means that we can build the computation graph first, with placeholder variables independently of specific input values, and then reuse it for different inputs. This is how many deep learning frameworks work.</font>

In [31]:
#I'll use the library functools for this
from functools import partial

In [32]:
class Variable: #Simple variable. These are the leafs of our tree, they can request to have a gradient calculated, or not
    def __init__(self, value):
        self.value = value
        self.gradRoutes = [] # A variable by default has no grad Routes (was not created by anything, just defined)
        self.grad = 0.0
    
    def backProp(self, route_val = 1.0):
        # Add together the incoming gradients from the different routes that lead to a node - this will be the final gradient for the node
        self.grad += route_val
                
        # For every parent variable and corresponding local derivative value that we have in the gradRoutes, continue with the gradient calculation
        for variable, local_derivative_value in self.gradRoutes:
            # Multiply the incoming gradient with the local derivatives corresponding the parent variable, and continue the backpropagation
            variable.backProp(local_derivative_value * route_val)

    def target_backprop(self,vars:list[Variable],route_val = 1.0)->float:
        # Add together the incoming gradients from the different routes that lead to a node - this will be the final gradient for the node
        self.grad += route_val
        if self in vars:
            vars.pop(self)
        if(len(vars) == 0):
            return None
        # For every parent variable and corresponding local derivative value that we have in the gradRoutes, continue with the gradient calculation
        for variable, local_derivative_value in self.gradRoutes:
            # Multiply the incoming gradient with the local derivatives corresponding the parent variable, and continue the backpropagation
            variable.backProp(local_derivative_value * route_val)
            
    def zeroGrad(self):
        self.grad = 0.0
        
    def zeroGradsRecursively(self):
        self.zeroGrad()
        for variable, _ in self.gradRoutes:
            variable.zeroGradsRecursively()
            
    def __add__(self, b): #called if left operand is of this class
        if isinstance(b,Variable):
            return partial(vAdd,(self, b))
        else:
            return partial(vAdd,(self, Variable(b)))
    
    def __radd__(self, b): #called if right operand is of this class
        return partial(vAdd,(self, Variable(b)))

    def __sub__(self, b):  #called if left operand is of this class
        if isinstance(b,Variable):
            return partial(vSub,(self, b))
        else:
            return partial(vSub,(self, Variable(b)))
    
    def __rsub__(self, b): #called if right operand is of this class
        return partial(vSub,(self, Variable(b)))

    def __mul__(self, b): 
        if isinstance(b,Variable):
            return partial(vMul,(self, b))
        else:
            return partial(vMul,(self, Variable(b)))

    def __rmul__(self, b): #called if right operand is of this class
        return partial(vMul,(self, Variable(b)))

    def __truediv__(self, b):
        if isinstance(b,Variable):
            return partial((vDiv,(self, b)))
        else:
            return partial(vDiv,(self, Variable(b)))
    
    def __rtruediv__(self, b): #called if right operand is of this class
        return partial(vDiv,(self, Variable(b)))

    def __pow__(self, b):
        if isinstance(b,Variable):
            return partial(vPow,(self,b))     
        else:
            return partial(vPow,(self, Variable(b)))
    
    def __rpow__(self, b): #called if right operand is of this class
        return partial(vPow,(self, Variable(b)))
    
    def __neg__(self):
        return partial(vNeg,(self))
        
    def __str__(self):
        return 'Value: {self.value}, Gradient: {self.grad}'.format(self=self)

In [33]:
a = Variable(4)   # a = 4
b = Variable(3)   # b = 3
c = Variable(8)
e = Variable(2)
res = 1 + a - b - 2
res.backProp()
recursive_print_grads(res)

TypeError: unsupported operand type(s) for -: 'functools.partial' and 'int'

In [56]:
#res = vExp(((a - 4 + (6**c)) * -0.5 ) / (c**(3)))# res = a * c = 28
res.backProp()
recursive_print_grads(res)

TypeError: unsupported operand type(s) for *: 'functools.partial' and 'float'

---
### <font color=blue>Excercise 5 (Easy):</font>
    
<font color=blue>Write some code to manually check that your gradient calculation is correct, using the property of:</font>

$$
f'(x) = \frac {f(x+\epsilon) - f(x-\epsilon)}{2 \epsilon}
$$

<font color=blue>where $\epsilon$ is a very small number to approximately calculate the gradient. Then use it to calculate the derivative of the function $f(x) = 21 * x^3$ at $x=3.2$. Double check that our framework gives you the same result.</font>

In [40]:
def fofx(x):
    return 21 * (x**3)

In [47]:
x = Variable(3.2)
res = fofx(x)
res.backProp()
def check_grad(res:Variable,e):
    fprime = (fofx(x+e)-fofx(x-e)) / (2*e)
    lambd = x.grad - fprime
    print("Test gradient: ",fprime.value)
    print("Framework grad: ",x.grad)

check_grad(res,0.1)


Test gradient:  645.3300000000007
Framework grad:  645.1200000000001
